In [ ]:
# default_exp core.classes

In [ ]:
# hide
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [ ]:
# hide
import warnings

from nbdev.export import *
from nbdev.showdoc import *

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

# Classes
> Interfaces common to all `Modules` and `Models` in Gale.

In [ ]:
# export
import copy
import math
from abc import ABC, ABCMeta, abstractmethod
from contextlib import contextmanager
from typing import *

import hydra
import pytorch_lightning as pl
import torch
from fastcore.all import L, noop, patch
from omegaconf import DictConfig, OmegaConf
from torch.nn import Module
import torchmetrics

from gale.config import BaseGaleConfig, DatasetsConfig, OptimizationConfig
from gale.config.optimizers import create_optimizer
from gale.core.logging import setup_logger
from gale.core.nn.utils import params, trainable_params

_logger = setup_logger()

<IPython.core.display.Javascript object>

In [ ]:
# export
class Configurable(ABC):
    @classmethod
    def from_config_dict(cls, config: DictConfig):
        """Instantiates object using `DictConfig-based` configuration"""
        # Resolve the config dict
        if isinstance(config, DictConfig):
            config = OmegaConf.to_container(config, resolve=True)
            config = OmegaConf.create(config)

        if "_target_" in config:
            # regular hydra-based instantiation
            instance = hydra.utils.instantiate(config=config)
        else:
            # instantiate directly using kwargs
            try:
                instance = cls(cfg=config)
            except:
                instance = cls(**config)

        if not hasattr(instance, "_cfg"):
            instance._cfg = config
        return instance

<IPython.core.display.Javascript object>

In [ ]:
# export
class GaleModule(Module, Configurable, metaclass=ABCMeta):
    """
    Abstract class offering interface which should be implemented by all `Backbones`,
    `Heads` and `Meta Archs` in gale.
    """

    @abstractmethod
    def forward(self) -> Any:
        """
        The main logic for the model lives here. Can return either features, logits
        or loss.
        """
        raise NotImplementedError

    @abstractmethod
    def build_param_dicts(self) -> Union[Iterable, List[Dict], Dict, List]:
        """
        Should return the iterable of parameters to optimize or dicts defining parameter groups
        for the Module.
        """
        raise NotImplementedError

    @property
    def param_lists(self):
        "Returns the list of paramters in the module"
        return [p for p in self.parameters()]

    def all_params(self, n=slice(None), with_grad=False):
        "List of `param_groups` upto n"
        res = L(p for p in self.param_lists[n])
        return (
            L(o for o in res if hasattr(o, "grad") and o.grad is not None)
            if with_grad
            else res
        )

    def _set_require_grad(self, rg, p):
        p.requires_grad_(rg)

    def unfreeze(self) -> None:
        """
        Unfreeze all parameters for training.
        """
        for param in self.parameters():
            param.requires_grad = True

        self.train()

    def freeze(self) -> None:
        """
        Freeze all params for inference & set model to eval
        """
        for param in self.parameters():
            param.requires_grad = False
        self.eval()

    def freeze_to(self, n) -> None:
        "Freeze parameter groups up to `n`"
        self.frozen_idx = n if n >= 0 else len(self.param_lists) + n
        if self.frozen_idx >= len(self.param_lists):
            _logger.warning(
                f"Freezing {self.frozen_idx} groups; model has {len(self.param_lists)}; whole model is frozen."
            )

        for o in self.all_params(slice(n, None)):
            self._set_require_grad(True, o)

        for o in self.all_params(slice(None, n)):
            self._set_require_grad(False, o)

    @contextmanager
    def as_frozen(self):
        """
        Context manager which temporarily freezes a module, yields control
        and finally unfreezes the module.
        """
        self.freeze()

        try:
            yield
        finally:
            self.unfreeze()

<IPython.core.display.Javascript object>

In [ ]:
show_doc(GaleModule)

<h2 id="GaleModule" class="doc_header"><code>class</code> <code>GaleModule</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>GaleModule</code>() :: `Module`

Abstract class offering interface which should be implemented by all `Backbones`,
`Heads` and `Meta Archs` in gale.

<IPython.core.display.Javascript object>

*Required Methods -*

In [ ]:
show_doc(GaleModule.forward)
show_doc(GaleModule.build_param_dicts)

<h4 id="GaleModule.forward" class="doc_header"><code>GaleModule.forward</code><a href="__main__.py#L8" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleModule.forward</code>()

The main logic for the model lives here. Can return either features, logits
or loss.

<h4 id="GaleModule.build_param_dicts" class="doc_header"><code>GaleModule.build_param_dicts</code><a href="__main__.py#L16" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleModule.build_param_dicts</code>()

Should return the iterable of parameters to optimize or dicts defining parameter groups
for the Module.

<IPython.core.display.Javascript object>

*Extra functionality -*

In [ ]:
show_doc(GaleModule.from_config_dict)
show_doc(GaleModule.param_lists)
show_doc(GaleModule.all_params)
show_doc(GaleModule.freeze)
show_doc(GaleModule.freeze_to)
show_doc(GaleModule.unfreeze)
show_doc(GaleModule.as_frozen)

<h4 id="Configurable.from_config_dict" class="doc_header"><code>Configurable.from_config_dict</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>Configurable.from_config_dict</code>(**`config`**:`DictConfig`)

Instantiates object using `DictConfig-based` configuration

<h4 id="GaleModule.param_lists" class="doc_header"><code>GaleModule.param_lists</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Returns the list of paramters in the module

<h4 id="GaleModule.all_params" class="doc_header"><code>GaleModule.all_params</code><a href="__main__.py#L29" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleModule.all_params</code>(**`n`**=*`slice(None, None, None)`*, **`with_grad`**=*`False`*)

List of `param_groups` upto n

<h4 id="GaleModule.freeze" class="doc_header"><code>GaleModule.freeze</code><a href="__main__.py#L50" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleModule.freeze</code>()

Freeze all params for inference & set model to eval

<h4 id="GaleModule.freeze_to" class="doc_header"><code>GaleModule.freeze_to</code><a href="__main__.py#L58" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleModule.freeze_to</code>(**`n`**)

Freeze parameter groups up to `n`

<h4 id="GaleModule.unfreeze" class="doc_header"><code>GaleModule.unfreeze</code><a href="__main__.py#L41" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleModule.unfreeze</code>()

Unfreeze all parameters for training.

<h4 id="GaleModule.as_frozen" class="doc_header"><code>GaleModule.as_frozen</code><a href="__main__.py#L72" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleModule.as_frozen</code>()

Context manager which temporarily freezes a module, yields control
and finally unfreezes the module.

<IPython.core.display.Javascript object>

In [ ]:
# export
class OptimSchedBuilder:
    """
    Interface that constructs an Optimizer and Scheduler for `GaleTask` from config.
    """

    _train_dl: Callable
    _trainer: pl.Trainer
    optimization_cfg: DictConfig

<IPython.core.display.Javascript object>

In [ ]:
# collapse-output
from dataclasses import dataclass, field

from fastcore.all import Path
from nbdev.export import Config
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST

from gale.config.optimizers import *
from gale.config.schedulers import *

# Create a mock config
opt_cfg = OptimizationConfig(AdamWConfig(lr=1e-03), OneCycleLRConfig(max_lr=1e-03))
cfg = BaseGaleConfig(optimization=opt_cfg, datasets=DatasetsConfig())
cfg = OmegaConf.structured(cfg)

data_path = Path(Config().path("nbs_path")) / "data"
dset = FashionMNIST(root=data_path, download=True)

builder = OptimSchedBuilder()
# create mock dataloaders and trainer for builder
builder._train_dl = DataLoader(dset, batch_size=32)
builder._trainer = pl.Trainer(max_epochs=10, accumulate_grad_batches=1)

print(f"Original Config: \n{OmegaConf.to_yaml(cfg.optimization)}")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Original Config: 
optimizer:
  _target_: torch.optim.adamw.AdamW
  lr: 0.001
  weight_decay: 0.01
  betas:
  - 0.95
  - 0.999
  eps: 1.0e-05
  amsgrad: false
scheduler:
  _target_: torch.optim.lr_scheduler.OneCycleLR
  max_lr: 0.001
  total_steps: null
  epochs: null
  steps_per_epoch: null
  pct_start: 0.3
  anneal_strategy: cos
  cycle_momentum: true
  base_momentum: 0.85
  max_momentum: 0.95
  div_factor: 25.0
  final_div_factor: 10000.0
  last_epoch: -1
interval: epoch
monitor: null
steps_per_epoch: null
max_steps: null
max_epochs: null



<IPython.core.display.Javascript object>

In [ ]:
# export
@patch
def prepare_optimization_config(self: OptimSchedBuilder, config: DictConfig):
    """
    Prepares `OptimizationConfig` config and adds some interval
    values and infers values like max_steps, max_epochs, etc. This method
    takes in a config that is inherited from `BaseGaleConfig`. This method also fills in
    the values for `max_iters` & `epochs`, `steps_per_epoch` which are required by some of
    the LearningRate Schedulers.
    """
    opt_config = copy.deepcopy(config)
    self.optimization_cfg = opt_config

    self.optimization_cfg["steps_per_epoch"] = len(self._train_dl)

    if self._trainer.max_epochs is None and self._trainer.max_steps is None:
        _logger.error(
            "Either one of max_epochs or max_epochs must be provided in Trainer"
        )
        raise ValueError

    if self._trainer.max_steps is None:
        # calculate max_steps
        self.optimization_cfg["max_epochs"] = self._trainer.max_epochs
        accumulate_grad_batches = self._trainer.accumulate_grad_batches
        limit_train_batches = self._trainer.limit_train_batches
        steps_per_epoch = self.optimization_cfg["steps_per_epoch"]

        if isinstance(limit_train_batches, int) or limit_train_batches == 0.0:
            steps_per_epoch = min(steps_per_epoch, int(limit_train_batches))
        elif steps_per_epoch != float("inf"):
            steps_per_epoch = int(steps_per_epoch * limit_train_batches)
            if accumulate_grad_batches == 1:
                steps_per_epoch = max(steps_per_epoch, 1)
        max_steps = (
            math.ceil(steps_per_epoch / accumulate_grad_batches)
            * self.optimization_cfg["max_epochs"]
        )

        self.optimization_cfg["max_steps"] = max_steps

    else:
        self.optimization_cfg["max_steps"] = self._trainer.max_steps
        # compute max epochs
        self.optimization_cfg["max_epochs"] = self._trainer.max_steps * len(
            self._train_dl
        )

    # covert config to Dictionary
    sched_config = OmegaConf.to_container(self.optimization_cfg.scheduler, resolve=True)

    # populate values in learning rate schedulers
    if "max_iters" in sched_config:
        if sched_config["max_iters"] == "???" or sched_config["max_iters"] is None:
            OmegaConf.update(
                self.optimization_cfg,
                "scheduler.max_iters",
                self.optimization_cfg["max_steps"],
            )

    if "epochs" in sched_config:
        if sched_config["epochs"] == "???" or sched_config["epochs"] is None:
            OmegaConf.update(
                self.optimization_cfg,
                "scheduler.epochs",
                self.optimization_cfg["max_epochs"],
            )

    if "steps_per_epoch" in sched_config:
        if (
            sched_config["steps_per_epoch"] == "???"
            or sched_config["steps_per_epoch"] is None
        ):
            OmegaConf.update(
                self.optimization_cfg,
                "scheduler.steps_per_epoch",
                self.optimization_cfg["steps_per_epoch"],
            )

<IPython.core.display.Javascript object>

In [ ]:
# collapse-output
builder.prepare_optimization_config(config=cfg.optimization)
print(f"Modified Optimization Config: \n{OmegaConf.to_yaml(builder.optimization_cfg)}")

Modified Optimization Config: 
optimizer:
  _target_: torch.optim.adamw.AdamW
  lr: 0.001
  weight_decay: 0.01
  betas:
  - 0.95
  - 0.999
  eps: 1.0e-05
  amsgrad: false
scheduler:
  _target_: torch.optim.lr_scheduler.OneCycleLR
  max_lr: 0.001
  total_steps: null
  epochs: 10
  steps_per_epoch: 1875
  pct_start: 0.3
  anneal_strategy: cos
  cycle_momentum: true
  base_momentum: 0.85
  max_momentum: 0.95
  div_factor: 25.0
  final_div_factor: 10000.0
  last_epoch: -1
interval: epoch
monitor: null
steps_per_epoch: 1875
max_steps: 18750
max_epochs: 10



<IPython.core.display.Javascript object>

In [ ]:
# export
@patch
def build_optimizer(
    self: OptimSchedBuilder, params: Union[Iterable, Any]
) -> torch.optim.Optimizer:
    """
    Builds a single optimizer from `OptimizationConfig`. `params` are the parameter
    dict with the weights for the optimizer to optimizer.

    Note this method must be called after `prepare_optimization_config()`
    """
    if not isinstance(self.optimization_cfg, DictConfig):
        _logger.warning(
            "optimization_cfg not found, did you call `prepare_optimization_config`."
        )
        raise NameError
    else:
        if (
            self.optimization_cfg.optimizer is None
            and self.optimization_cfg.optimizer._target_ is None
        ):
            _logger.warning("Optimizer is None, so no optimizer will be created.")
            opt = None
        else:
            opt = create_optimizer(self.optimization_cfg.optimizer, params=params)
        return opt

<IPython.core.display.Javascript object>

In [ ]:
params = [torch.nn.Parameter(torch.randn(1, 2))]
optimizer = builder.build_optimizer(params)
assert isinstance(optimizer, torch.optim.AdamW)

<IPython.core.display.Javascript object>

In [ ]:
# export
@patch
def build_lr_scheduler(
    self: OptimSchedBuilder, optimizer: torch.optim.Optimizer
) -> Union[None, Dict]:
    """
    Builds a LearningRate scheduler from `OptimizationConfig`. Returns an LRScheduler dict
    that is required by PyTorch Lightning for LRSchedulers.
    Note this method must be called after `prepare_optimization_config()`
    """
    if not isinstance(self.optimization_cfg, DictConfig):
        _logger.warning(
            "optimization_cfg not found, did you call `prepare_optimization_config`."
        )
        raise NameError
    else:
        if (
            self.optimization_cfg.scheduler._target_ is None
            and self.optimization_cfg.scheduler is None
        ):
            _logger.warning("scheduler is None, so no scheduler will be created.")
            sched = None
        else:
            _temp = OmegaConf.to_container(
                self.optimization_cfg.scheduler, resolve=True
            )
            kwds = {}
            c_sch = {}

            # if a key value is ListConfig then we convert it to simple list
            for key, value in _temp.items():
                if isinstance(value, list):
                    kwds[key] = list(value)
                else:
                    c_sch[key] = value

            sched = hydra.utils.instantiate(c_sch, optimizer=optimizer, **kwds)

            # convert the lr_scheduler to pytorch-lightning LRScheduler dictionary format
            sched = dict(
                scheduler=sched,
                interval=self.optimization_cfg.interval,
                monitor=self.optimization_cfg.monitor,
            )
            return sched

<IPython.core.display.Javascript object>

In [ ]:
scheduler = builder.build_lr_scheduler(optimizer)
assert isinstance(scheduler, Dict)
assert isinstance(scheduler["scheduler"], torch.optim.lr_scheduler.OneCycleLR)
print(scheduler)

{'scheduler': <torch.optim.lr_scheduler.OneCycleLR object at 0x7ffd2c1f4630>, 'interval': 'epoch', 'monitor': None}


<IPython.core.display.Javascript object>

In [ ]:
# export
class GaleTask(pl.LightningModule, OptimSchedBuilder, metaclass=ABCMeta):
    """
    Interface for Pytorch-lightning based Gale modules
    """

    def __init__(
        self,
        cfg: DictConfig,
        trainer: Optional[pl.Trainer] = None,
        metrics: Union[torchmetrics.Metric, Mapping, Sequence, None] = None,
    ):
        """
        Base class from which all NeMo models should inherit.

        Arguments:
        1. `cfg` `(DictConfig)`:  configuration object. cfg object should be inherited from `BaseGaleConfig`.
        2. `trainer` `(Optional, pl.Trainer)`: Pytorch Lightning Trainer instance
        3. `metrics` `(Optional)`: Metrics to compute for training and evaluation.
        """
        super().__init__()
        self._cfg = OmegaConf.structured(cfg)

        self.save_hyperparameters(self._cfg)
        self._train_dl = noop
        self._validation_dl = noop
        self._test_dl = noop
        self._optimizer = noop
        self._scheduler = noop
        self._trainer = trainer
        self.metrics = metrics

    def train_dataloader(self) -> torch.utils.data.DataLoader:
        "Returns the Dataloader used for Training"
        if self._train_dl is not None and self._train_dl is not noop:
            return self._train_dl

    def val_dataloader(self) -> Any:
        "Returns the List of Dataloaders or Dataloader used for Validation"
        if self._validation_dl is not None and self._validation_dl is not noop:
            return self._validation_dl

    def test_dataloader(self) -> Any:
        "Returns the List of Dataloaders or Dataloader used for Testing"
        if self._test_dl is not None and self._test_dl is not noop:
            return self._test_dl

    @abstractmethod
    def forward(self, x: torch.Tensor):
        """
        The Forward method for LightningModule, users should modify this method.
        """
        raise NotImplementedError

    @abstractmethod
    def setup_training_data(self, train_data_config: Union[DictConfig, Dict]):
        """
        Setups data loader to be used in training

        Arguments:
        1. `train_data_config`: training data loader parameters.
        """
        raise NotImplementedError

    @abstractmethod
    def setup_validation_data(self, val_data_config: Union[DictConfig, Dict]):
        """
        Setups data loader to be used in validation

        Arguments:
        1. `val_data_config`: validation data loader parameters.
        """
        raise NotImplementedError

    def setup_test_data(
        self, test_data_config: Optional[Union[DictConfig, Dict]] = None
    ):
        """
        (Optionally) Setups data loader to be used in test

        Arguments:
        1. `test_data_config`: test data loader parameters.
        """
        raise NotImplementedError

    @property
    def param_dicts(self) -> Union[Iterator, List[Dict]]:
        """
        Property that returns the param dicts for optimization.
        Override for custom training behaviour. Currently returns all the trainable paramters.
        """
        return L(self).map(trainable_params)

<IPython.core.display.Javascript object>

In [ ]:
show_doc(GaleTask)

show_doc(GaleTask.train_dataloader)
show_doc(GaleTask.setup_training_data)

show_doc(GaleTask.val_dataloader)
show_doc(GaleTask.setup_validation_data)

show_doc(GaleTask.test_dataloader)
show_doc(GaleTask.setup_test_data)

<h2 id="GaleTask" class="doc_header"><code>class</code> <code>GaleTask</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>GaleTask</code>(**`cfg`**:`DictConfig`, **`trainer`**:`Optional`\[`Trainer`\]=*`None`*, **`metrics`**:`Union`\[`Metric`, `Mapping`, `Sequence`, `NoneType`\]=*`None`*) :: `LightningModule`

Interface for Pytorch-lightning based Gale modules

<h4 id="GaleTask.train_dataloader" class="doc_header"><code>GaleTask.train_dataloader</code><a href="__main__.py#L33" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.train_dataloader</code>()

Returns the Dataloader used for Training

<h4 id="GaleTask.setup_training_data" class="doc_header"><code>GaleTask.setup_training_data</code><a href="__main__.py#L55" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.setup_training_data</code>(**`train_data_config`**:`Union`\[`DictConfig`, `Dict`\])

Setups data loader to be used in training

Arguments:
1. `train_data_config`: training data loader parameters.

<h4 id="GaleTask.val_dataloader" class="doc_header"><code>GaleTask.val_dataloader</code><a href="__main__.py#L38" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.val_dataloader</code>()

Returns the List of Dataloaders or Dataloader used for Validation

<h4 id="GaleTask.setup_validation_data" class="doc_header"><code>GaleTask.setup_validation_data</code><a href="__main__.py#L65" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.setup_validation_data</code>(**`val_data_config`**:`Union`\[`DictConfig`, `Dict`\])

Setups data loader to be used in validation

Arguments:
1. `val_data_config`: validation data loader parameters.

<h4 id="GaleTask.test_dataloader" class="doc_header"><code>GaleTask.test_dataloader</code><a href="__main__.py#L43" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.test_dataloader</code>()

Returns the List of Dataloaders or Dataloader used for Testing

<h4 id="GaleTask.setup_test_data" class="doc_header"><code>GaleTask.setup_test_data</code><a href="__main__.py#L75" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.setup_test_data</code>(**`test_data_config`**:`Union`\[`DictConfig`, `Dict`, `NoneType`\]=*`None`*)

(Optionally) Setups data loader to be used in test

Arguments:
1. `test_data_config`: test data loader parameters.

<IPython.core.display.Javascript object>

In [ ]:
show_doc(GaleTask.forward)

show_doc(GaleTask.param_dicts)

<h4 id="GaleTask.forward" class="doc_header"><code>GaleTask.forward</code><a href="__main__.py#L48" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.forward</code>(**`x`**:`Tensor`)

The Forward method for LightningModule, users should modify this method.

<h4 id="GaleTask.param_dicts" class="doc_header"><code>GaleTask.param_dicts</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Property that returns the param dicts for optimization.
Override for custom training behaviour. Currently returns all the trainable paramters.

<IPython.core.display.Javascript object>

In [ ]:
# export
@patch
def shared_step(self: GaleTask, batch: Any, batch_idx: int, stage: str) -> Any:
    """
    The common training/validation/test step. Override for custom behavior. This step
    is shared between training/validation/test step. For training/validation/test steps
    `stage` is train/val/test respectively. You training logic should go here avoid directly overriding
    training/validation/test step methods.
    """
    raise NotImplementedError

<IPython.core.display.Javascript object>

In [ ]:
# export
@patch
def training_step(self: GaleTask, batch: Any, batch_idx: int) -> Any:
    """
    The training step of the LightningModule. For common use cases you need
    not need to override this method. See `GaleTask.shared_step()`
    """
    return self.shared_step(batch, batch_idx, stage="train")

<IPython.core.display.Javascript object>

In [ ]:
# export
@patch
def validation_step(self: GaleTask, batch: Any, batch_idx: int) -> None:
    """
    The validation step of the LightningModule. For common use cases you need
    not need to override this method. See `GaleTask.shared_step()`
    """
    return self.shared_step(batch, batch_idx, stage="val")

<IPython.core.display.Javascript object>

In [ ]:
# export
@patch
def test_step(self: GaleTask, batch: Any, batch_idx: int) -> None:
    """
    The test step of the LightningModule. For common use cases you need
    not need to override this method. See `GaleTask.shared_step()`
    """
    return self.shared_step(batch, batch_idx, stage="test")

<IPython.core.display.Javascript object>

In [ ]:
show_doc(GaleTask.shared_step)
show_doc(GaleTask.training_step)
show_doc(GaleTask.validation_step)
show_doc(GaleTask.test_step)

<h4 id="GaleTask.shared_step" class="doc_header"><code>GaleTask.shared_step</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.shared_step</code>(**`batch`**:`Any`, **`batch_idx`**:`int`, **`stage`**:`str`)

The common training/validation/test step. Override for custom behavior. This step
is shared between training/validation/test step. For training/validation/test steps
`stage` is train/val/test respectively. You training logic should go here avoid directly overriding
training/validation/test step methods.

<h4 id="GaleTask.training_step" class="doc_header"><code>GaleTask.training_step</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.training_step</code>(**`batch`**:`Any`, **`batch_idx`**:`int`)

The training step of the LightningModule. For common use cases you need
not need to override this method. See [`GaleTask.shared_step()`](/gale/core.classes.html#GaleTask.shared_step())

<h4 id="GaleTask.validation_step" class="doc_header"><code>GaleTask.validation_step</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.validation_step</code>(**`batch`**:`Any`, **`batch_idx`**:`int`)

The validation step of the LightningModule. For common use cases you need
not need to override this method. See [`GaleTask.shared_step()`](/gale/core.classes.html#GaleTask.shared_step())

<h4 id="GaleTask.test_step" class="doc_header"><code>GaleTask.test_step</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.test_step</code>(**`batch`**:`Any`, **`batch_idx`**:`int`)

The test step of the LightningModule. For common use cases you need
not need to override this method. See [`GaleTask.shared_step()`](/gale/core.classes.html#GaleTask.shared_step())

<IPython.core.display.Javascript object>

In [ ]:
# export
@patch
def setup_optimization(
    self: GaleTask,
    optim_config: Optional[Union[DictConfig, Dict, OptimizationConfig]] = None,
):
    """
    Prepares an optimizer from a string name and its optional config parameters.

    Args:
    1. `optim_config`: A `dictionary`/`DictConfig` or instance of `OptimizationConfig`.
    """
    # If config was not explicitly passed to us
    if optim_config is None:
        # See if internal config has `optim` namespace
        if self._cfg is not None and hasattr(self._cfg, "optimization"):
            optim_config = self._cfg.optimization

    # If config is still None, or internal config has no Optim, return without instantiation
    if optim_config is None:
        _logger.info("No optimizer config provided, therefore no optimizer was created")
        return

    else:
        # Preserve the configuration
        if not isinstance(optim_config, DictConfig):
            optim_config = OmegaConf.create(optim_config)

        # prepare the optimization config
        self.prepare_optimization_config(optim_config)

        # Setup optimizer and scheduler
        self._optimizer = self.build_optimizer(self.param_dicts)
        self._scheduler = self.build_lr_scheduler(self._optimizer)


@patch
def configure_optimizers(self: GaleTask):
    """
    Choose what optimizers and learning-rate schedulers to use in your optimization.
    See https://pytorch-lightning.readthedocs.io/en/latest/common/optimizers.html
    """
    # if self.setup_optimization() has been called manually no
    # need to call again
    if self._optimizer is noop and self._scheduler is noop:
        self.setup_optimization()

    if self._scheduler is None:
        return self._optimizer
    else:
        return [self._optimizer], [self._scheduler]

<IPython.core.display.Javascript object>

In [ ]:
show_doc(GaleTask.setup_optimization)
show_doc(GaleTask.configure_optimizers)

<h4 id="GaleTask.setup_optimization" class="doc_header"><code>GaleTask.setup_optimization</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.setup_optimization</code>(**`optim_config`**:`Union`\[`DictConfig`, `Dict`, [`OptimizationConfig`](/gale/config.common.html#OptimizationConfig), `NoneType`\]=*`None`*)

Prepares an optimizer from a string name and its optional config parameters.

Args:
1. `optim_config`: A `dictionary`/`DictConfig` or instance of [`OptimizationConfig`](/gale/config.common.html#OptimizationConfig).

<h4 id="GaleTask.configure_optimizers" class="doc_header"><code>GaleTask.configure_optimizers</code><a href="__main__.py#L37" class="source_link" style="float:right">[source]</a></h4>

> <code>GaleTask.configure_optimizers</code>()

Choose what optimizers and learning-rate schedulers to use in your optimization.
See https://pytorch-lightning.readthedocs.io/en/latest/common/optimizers.html

<IPython.core.display.Javascript object>

## Export -

In [ ]:
# hide
notebook2script()

Converted 00_core.logging.ipynb.
Converted 00a_core.structures.ipynb.
Converted 00b_core.visualize.ipynb.
Converted 01_core.nn.utils.ipynb.
Converted 01a_core.nn.losses.ipynb.
Converted 01b_core.nn.optim.optimizers.ipynb.
Converted 01c_core.nn.optim.lr_schedulers.ipynb.
Converted 02_config.optimizers.ipynb.
Converted 02a_config.schedulers.ipynb.
Converted 02b_config.common.ipynb.
Converted 03_core.classes.ipynb.
Converted 05_collections.pandas.ipynb.
Converted 06a_collections.callbacks.notebook.ipynb.
Converted 06b_collections.callbacks.ema.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>